# Preeminence

Welcome to the world conquest game, played by proxy!

_While we're waiting, start here: **https://aka.ms/preem**_

![Classic Map](https://douglasorr.github.io/Preeminence/img/eg_classic.svg)

## Format

Two ways to "enjoy" this - as a workshop / as a challenge...

 - **Tue 1100-1300** - intro workshop
   - 30 mins - intro
   - 90 mins - tutorial / development
 - **Thurs 1700** - challenge entry submission deadline
 - **Fri ~1700** - winners announced

### Workshop format

 - intro presentation
   - rules of the game
   - creating & debugging an agent
   - evaluating & scaling up
   - strategy tips
 - tutorial / open development

## 1. Today's Terrific Task To Try Thy Tactical Tenacity...

Write an _agent_ to play a territory conquest game in a graph (called a _map_). The goal is to go from this:

![game start](img/mini/start.svg)

...to this (if you're the purple agent, that is):

![game end](img/mini/end.svg)

Let's step through, _action_ by _action_...

![frame_0](img/mini/step_0.svg)

Starting territories have already been pre-assigned, and we're in the initial _placement_ phase.

![frame_0](img/mini/step_0.svg)

Each agent takes turns placing one army on one owned territory.

![frame_1](img/mini/step_1.svg)

Each agent takes turns placing one army on one owned territory.

![frame_2](img/mini/step_2.svg)

Each agent takes turns placing one army on one owned territory.

![frame_3](img/mini/step_3.svg)

(Fast forward) - all agents have now placed all of their initial armies.

![frame_8](img/mini/step_8.svg)

The purple agent starts their _turn_ by _reinforcing_ an owned territory.

![frame_9](img/mini/step_9.svg)

Then _attacking_ an adjacent enemy territory.

![frame_10](img/mini/step_10.svg)

And another.

![frame_11](img/mini/step_11.svg)

And another (until its bloodlust is sated)!

![frame_12](img/mini/step_12.svg)

(Fast forward) - purple finishes their turn with a _move_ (owned => owned).

![frame_15](img/mini/step_15.svg)

Green reinforces.

![frame_16](img/mini/step_16.svg)

Green attacks.

![frame_17](img/mini/step_17.svg)

(Fast forward) - purple reinforces with bonus armies for controlling a continent at the beginning of its turn.

![frame_24](img/mini/step_24.svg)

(Fast forward) - purple eventually wins by conquering the whole map.

![frame_26](img/mini/step_26.svg)
![end](img/mini/end.svg)

In [ ]:
import IPython.display

In [ ]:
# Let's watch the whole game
IPython.display.Video('https://douglasorr.github.io/Preeminence/img/mini/game.mp4')

## 2. Advice Arising About Agent Accreditation

Soon, the **tutorial** will guide you through how to create & debug an Agent, in (possibly painful) detail. But here's a quick overview...

```python
import preem as P

class MyAgent(P.Agent):
    def place(self, state):
        pass  # TODO
    def redeem(self, state):
        pass  # TODO
    def reinforce(self, state, count):
        pass  # TODO
    def act(self, state, earned_card):
        pass  # TODO
```

In [ ]:
import preem as P
import agents.random_agent
import random
RandomAgent = agents.random_agent.Agent
random.seed(10)

In [ ]:
P.Game.play(P.Map.load('maps/mini.json'), [RandomAgent()] * 3)

Those 4 methods are called in this order:

![agent flow](img/agent_flow.svg)

 - Game assigns starting territories randomly to each player,
   - then calls `start_game` (an optional override) to allow the Agent to set itself up.
 - Game calls `place` repeatedly, for each player in turn, to place a single army on an owned territory,
   until enough armies have been placed.
 - For each turn, game:
   - calls `redeem` to decide if any cards should be declared for bonus armies,
   - calls `reinforce` to place multiple armies on one or more owned territories,
   - calls `act` repeatedly, until it returns a turn-ending action (such as a move).

Key abstractions:

 - `Agent` - your code
 - `PlayerState` (often just `state`) - player specific & private data
   - contains `World` - visible game data
     - contains `Map` - fixed layout
 - `Event` - logs an agent's method call response & `state`

In [ ]:
random.seed(42)
game = P.Game.start(P.Map.load('maps/mini.json'), [RandomAgent()] * 3)

event = game.next_event(method='act', player_index=0)
print(event.state.map)
print('Owners: ', event.state.world.owners)
print('Armies: ', event.state.world.armies)
print('Action: ', event.result)

## 3. Previewing Possibly Predictive Performance

We've seen `Game.play`, `Game.start` (and actually `Game.watch`), which are **single game** & great for **developing & debugging**.

We've also been on a small map `mini`, quick & easy to debug, but not a great evaluation.

But to really **evaluate** progress, you'll want to run a `Tournament` (multiple games, in parallel) on `classic`...

In [ ]:
P.Tournament.run(P.Map.load('maps/classic.json'), [RandomAgent()] * 3, rounds=100)

In [ ]:
MAPS_GRID = IPython.display.HTML("""
<table>
  <tr>
    <td>tiny3</td>
    <td><img src="img/maps/tiny3.svg" width="200px"/></td>
    <td>mini</td>
    <td><img src="img/maps/mini.svg" width="200px"/></td>
  </tr>
  <tr>
    <td>tiny4</td>
    <td><img src="img/maps/tiny4.svg" width="200px"/></td>
    <td>quad</td>
    <td><img src="img/maps/quad.svg" width="200px"/></td>
  </tr>
  <tr>
    <td>classic</td>
    <td align="center" colspan="3"><img src="img/maps/classic.svg" width="400px"/></td>
  </tr>
</table>
""")

In [ ]:
MAPS_GRID

# 4. Savouring Some Seemingly Solid Strategies

Some vague ideas to get you started...

### The top-level approach

**Strategic agent** - encodes the rules & strategies you would use.

**Machine learning agent** - optimized/tuned to win, from lots of up-front games.

**Adaptive agent** - adapts its behaviour "online" based on the game or games played.

### Offensive vs defensive

_Note that for a given attack, dice favour the attacker (assuming 3 vs 2), but only slightly._

**Offensive agent** - acquires territory quickly, leaving some easy pickings.

**Defensive agent** - aims to expand slowly and steadily, defending what has been won.

### Landlocking

Landlocked armies are surrounded by your own territories, so are unable to attack.

They can only move at one territory per turn, and can't immediately help defend or attack, so are less efficient.

### No mercy

Knocking out other players is very valuable, as you get to claim their cards.

Do I have enough armies to go for the jugular? What is the best path to take?

### Continental focus

Continents can, when held, be a great steady source of armies.

Some are easy to hold - Australia, South America - will you opt in to fierce early game action?

## 5. Go! https://aka.ms/preem

In [ ]:
IPython.display.Video('https://douglasorr.github.io/Preeminence/img/eg_classic.mp4')